In [2]:
import os, pandas as pd
from datetime import date, timedelta
from dotenv import load_dotenv
import finnhub

load_dotenv()
FINNHUB_KEY = os.getenv("FINNHUB_API_KEY")
if not FINNHUB_KEY:
    raise RuntimeError("Missing FINNHUB_API_KEY in .env")

client = finnhub.Client(api_key=FINNHUB_KEY)

def finnhub_company_news(symbol="AAPL", days_back=7):
    end = date.today()
    start = end - timedelta(days=days_back)
    news = client.company_news(symbol, _from=str(start), to=str(end))
    if not news:
        print(f"No company news for {symbol} in last {days_back} days.")
        return pd.DataFrame()
    df = pd.DataFrame([{
        "symbol": symbol,
        "source": n.get("source"),
        "headline": n.get("headline"),
        "summary": n.get("summary"),
        "datetime": pd.to_datetime(n.get("datetime"), unit="s", utc=True),
        "url": n.get("url")
    } for n in news])
    return df.sort_values("datetime").reset_index(drop=True)

df_company = finnhub_company_news("NVDA", days_back=7)
df_company.head()


,symbol,source,headline,summary,datetime,url
0,NVDA,Yahoo,These Are The Best Robinhood Stocks To Buy Or ...,"Buying a stock is easy, but purchasing the rig...",2021-04-25 04:33:14+00:00,https://finnhub.io/api/news?id=d46fd19800db2c1...
1,NVDA,Yahoo,IBM Is America’s Worst Tech Company,"IBM announced terrible quarterly results, and ...",2025-10-23 14:15:32+00:00,https://finnhub.io/api/news?id=9a4bbb5a3c1864f...
2,NVDA,Yahoo,Will The NEAR Protocol Survive After This Crit...,It’s hard to talk about crypto AI without ment...,2025-10-23 14:22:19+00:00,https://finnhub.io/api/news?id=072b41fb470ba0a...
3,NVDA,Yahoo,Nvidia Stock: All Eyes On Magnificent Seven Sp...,Nvidia stock edged higher and tested a key lev...,2025-10-23 14:22:20+00:00,https://finnhub.io/api/news?id=7a5fc53c66424f7...
4,NVDA,Yahoo,Experience Horizon Quantum’s Full Stack Powerh...,Download the Complete Report Here By Brandon H...,2025-10-23 15:05:15+00:00,https://finnhub.io/api/news?id=0ab592ac0469ee9...


In [3]:
def finnhub_general_news(category="general", limit=20):
    """
    category ∈ {"general","forex","crypto","merger"}
    """
    items = client.general_news(category=category, min_id=0)
    rows = [{
        "category": category,
        "source": x.get("source"),
        "headline": x.get("headline"),
        "summary": x.get("summary"),
        "datetime": pd.to_datetime(x.get("datetime"), unit="s", utc=True),
        "url": x.get("url")
    } for x in items[:limit]]
    return pd.DataFrame(rows).sort_values("datetime").reset_index(drop=True)

df_general = finnhub_general_news("general", limit=10)
df_general.head()


,category,source,headline,summary,datetime,url
0,general,MarketWatch,I’m 30 and worried I’ve been playing it too sa...,If you give money to a friend to build a busin...,2025-10-29 19:54:00+00:00,https://www.marketwatch.com/story/im-30-and-wo...
1,general,MarketWatch,Why ServiceNow is planning a stock split — and...,"ServiceNow’s stock price has hovered near $1,0...",2025-10-29 20:30:00+00:00,https://www.marketwatch.com/story/why-servicen...
2,general,MarketWatch,Starbucks puts up a surprise same-store sales ...,Shares of Starbucks Corp. rose in extended tra...,2025-10-29 20:36:00+00:00,https://www.marketwatch.com/story/starbucks-pu...
3,general,MarketWatch,"Dow’s climb toward 48,000 closing level is thw...",Wednesday’s climb toward a closing level above...,2025-10-29 20:48:00+00:00,https://www.marketwatch.com/story/dows-climb-t...
4,general,MarketWatch,Here’s why Chipotle is cutting its outlook — a...,Chipotle pulls back on its expectations for th...,2025-10-29 20:54:00+00:00,https://www.marketwatch.com/story/heres-why-ch...


In [3]:
def finnhub_news_sentiment(symbol="AAPL"):
    data = client.news_sentiment(symbol)
    # returns aggregates like buzz, sentiment score, articles count
    return pd.json_normalize(data)

finnhub_news_sentiment("AAPL")


FinnhubAPIException: FinnhubAPIException(status_code: 403): You don't have access to this resource.